In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import os
import sys
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

In [5]:
import jax
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt
import evosax

import json
import math
import drawsvg

In [6]:
import xylo
import xylo.types as t
import xylo.woods
import xylo.cut

import xylo.opt

import xylo.tuning
import xylo.database

import xylo.just

import xylo.resonators as r

In [7]:
layout = xylo.just.bass_partch

In [13]:
c = r.Config(radius = 0.090/2)
c

Config(temp=20, correction=0.61, radius=0.045)

In [14]:
slack = 0.05
tube_lengths = { k: r.length_of_freq(f, c) + slack for k,f in layout.get_freqs().items() }

In [15]:
sum([v for v in tube_lengths.values() if v > 0.00])

8.304666333352966

In [16]:
tube_lengths

{'A#3': 0.39778464578997946,
 'A3': 0.41168222526368253,
 'F#3': 0.5001213673690649,
 'F3': 0.5150454725993482,
 'D#3': 0.5697671917770535,
 'C#3': 0.6244889109547588,
 'A2': 0.7886540684878749,
 'G2': 0.8980975068432856,
 'E2': 1.0732070082119427,
 'D2': 1.1899466757910475,
 'C2': 1.3358712602649285}

In [12]:
for k, v in tube_lengths.items():
  if v >= 0.08:
    print(f"{k}: {v}m ({layout.get_freqs()[k]}Hz)")

A#3: 0.3886346457899794m (228.66666666666669Hz)
A3: 0.4025322252636825m (220.5Hz)
F#3: 0.4909713673690648m (179.66666666666666Hz)
F3: 0.5058954725993482m (174.2222222222222Hz)
D#3: 0.5606171917770535m (156.8Hz)
C#3: 0.6153389109547588m (142.54545454545456Hz)
A2: 0.7795040684878749m (112.0Hz)
G2: 0.8889475068432856m (98.0Hz)
E2: 1.0640570082119427m (81.66666666666667Hz)
D2: 1.1807966757910475m (73.5Hz)
C2: 1.3267212602649285m (65.33333333333333Hz)


In [10]:
def mkres(lens, filename, offsetx = 1.3):
  d = drawsvg.Drawing('297mm', '210mm')
  d.append(drawsvg.Rectangle(0, 0, '297mm', '210mm', stroke='green', stroke_width=2, fill='none'))
  d.append(drawsvg.Text(filename, '5mm', '20mm', '8mm'))
  grp = drawsvg.Group()
  y = 10
  for k,v in lens.items():
    mm = v * 1000 + offsetx
    print(k,v)
    grp.append(drawsvg.Text(f'{k}', '2mm', f'{mm-1}mm', f'{y + 3}mm'))
    grp.append(drawsvg.Circle(f'{mm}mm', f'{y + 1}mm', '0.5mm'))
    grp.append(drawsvg.Rectangle(f'{offsetx}mm', f'{y}mm', f'{mm}mm', f'0.5mm', stroke='red', stroke_width=1))
  grp.append(drawsvg.Circle(f'{offsetx}mm', f'{y+1}mm', '1mm'))

  d.append(grp)

  d.save_html(f'data/layout/{filename}.html')
  return d

mkres(tube_lengths, 'dia-resonator-lengths')

A1 0.07576761394742207
B1 0.08461152815796032
B2 0.08571701743427759
C1 0.09850910763166326
C2 0.09677191019745039
C3 0.09566642092113312
D1 0.10893229223694047
D2 0.11414388453957908
D3 0.10893229223694047
D4 0.10561582440798863
E1 0.12352475068432857
E2 0.1271728652961756
E3 0.12977866144749486
E4 0.12109267427643054
E5 0.12551463138169966
F1 0.1454134383554107
F2 0.1454134383554107
F3 0.1454134383554107
F4 0.1454134383554107
F5 0.1454134383554107
F6 0.1454134383554107
G1 0.17277429794426336
G2 0.16730212602649283
G3 0.1636540114146458
G4 0.17668299217124234
G5 0.16973420243439083
H1 0.20013515753311606
H2 0.189190813697575
H3 0.20013515753311606
H4 0.20795254598707397
I1 0.2274960171219687
I2 0.23296818903973926
I3 0.2366163036515863
J1 0.2822177362996741
J2 0.27674556438190356
K1 0.33693945547737936
